In [ ]:
import pandas as pd
from kafka import KafkaProducer, KafkaConsumer
import json
from json import dumps,loads
from s3fs import S3FileSystem
from time import sleep


make sure your kafka zookeper, server, producer and consumer is running in the background

In [ ]:
raw_data = "https://raw.githubusercontent.com/darshilparmar/stock-market-kafka-data-engineering-project/main/indexProcessed.csv"

df = pd.read_csv(raw_data)
df.head()

In [ ]:
producer = KafkaProducer(
    bootstrap_servers=["<MY_PUBLIC_IP>:9092"], #add your IP here
    value_serializer=lambda x: dumps(x).encode("utf-8"),
)

In [ ]:
consumer = KafkaConsumer(
    'demo_test',
     bootstrap_servers=['<MY_PUBLIC_IP>:9092'], #add your IP here
    value_deserializer=lambda x: loads(x.decode('utf-8')))

In [ ]:
# simulate stremaing
i = 0
while True:
    # randomly sample and convert to json
    dict_stock = df.sample(1).to_dict(orient="records")[0]
    producer.send("demo_test", value=dict_stock)
    print(f"{i} rows streamed")
    sleep(1)
    i += 1

In [ ]:
s3 = S3FileSystem()
s3_path = "s3://<S3_BUCKET_NAME>"

for count, i in enumerate(consumer):

    with s3.open(f"{s3_path}/stock_market_{count}.json", 'w') as file:
        print(f"Published {count} rows to S3")
        json.dump(i.value, file)    